In [1]:
import numpy as np
import scipy.interpolate as interpolate

In [16]:
env = ['duck_lift', 'banana_pass1', 'hammer_strike', 'mug_drink3', 'toothpaste_lift', 'cup_pour1'][-3]

def load_motion(motion_file):
    motion_file = np.load(motion_file, allow_pickle=True)
    reference_motion =  {k:v for k, v in motion_file.items()}
    reference_motion['s_0'] = reference_motion['s_0'][()]
    return reference_motion
data = load_motion(f'{env}.npz')
data.keys()

dict_keys(['s_0', 'object_translation', 'object_orientation', 'length', 'SIM_SUBSTEPS', 'DATA_SUBSTEPS'])

In [17]:
def data_analysis(data):
    print('range of data: ', np.min(data, axis=0), np.max(data, axis=0))
    print('start and end: ', data[0], data[-1])

In [18]:
print(data['object_translation'].shape)
trans_data = data['object_translation']  # x, y, z
data_analysis(trans_data)

(76, 3)
range of data:  [-0.07378107 -0.22652164  0.04697111] [-0.01605517  0.02340943  0.6736777 ]
start and end:  [-0.01605517  0.00845744  0.04939834] [-0.02621955 -0.00955264  0.04697111]


In [19]:
print(data['object_orientation'].shape)
ori_data = data['object_orientation']
data_analysis(ori_data)

(76, 4)
range of data:  [ 0.68115635 -0.07679475 -0.09302294 -0.2550657 ] [ 0.99861989  0.69032746  0.07249257 -0.04540542]
start and end:  [ 0.99861989 -0.01542553 -0.02141777 -0.04540542] [ 9.96369572e-01  9.83880363e-04 -1.70591632e-03 -8.51104980e-02]


In [20]:
def interpolate_data(data, points=None, initial_point=None, random_sample=5):
    if points is None:
        points = np.random.uniform(np.min(data), np.max(data), size=random_sample)
    if initial_point is not None: # set the same initial point as original traj
        points[0] = initial_point
    else:
        points[0] = data[0]
    len_points = points.shape[0]
    f = interpolate.interp1d(np.arange(len_points), points, kind='quadratic', fill_value="extrapolate")  # ‘linear’, ‘nearest’, ‘nearest-up’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, or ‘next’. 

    x_new = np.arange(0, len_points-1, (len_points-1)/data.shape[0])
    y_new = f(x_new)
    y_new[-1] = points[-1]
    return y_new

In [21]:
import copy
new_trans_data = []
new_points = np.array([copy.deepcopy(data['object_translation'][0]) for _ in range(3)])

# set offset for the stating point (original 0,0,0)
# considering the range of robot hand: TCDM/tcdm/envs/assets/robots/adroit/actuators.xml
# x: [-0.25, 0.25], y: [-0.2, 0.1], z: [-0.3, 0.5]
offset = [-0.2, 0.1, 0]
# offset = [0., 0., 0]

data['s_0']['motion_planned']['position'][30:33] += offset  # obj: (x, y, z)
data['s_0']['motion_planned']['position'][0] -= offset[0]  # hand: (-x, z, y)
data['s_0']['motion_planned']['position'][1] += offset[2]  # hand: (-x, z, y)
data['s_0']['motion_planned']['position'][2] += offset[1]  # hand: (-x, z, y)

# no need to change this
# data['s_0']['initialized']['position'][30:33] += offset  # obj: (x, y, z)
# data['s_0']['initialized']['position'][0] -= offset[0]  # hand: (-x, z, y)
# data['s_0']['initialized']['position'][1] += offset[2]  # hand: (-x, z, y)
# data['s_0']['initialized']['position'][2] += offset[1]  # hand: (-x, z, y)

# set points for arbirary trajectory
new_points[1][2] += 0.4 # z
new_points[2][1] += 0.2 # y
new_points[2][2] += -0.1 # z

new_points[:] += offset


print(new_points)
for i, (d, nd) in enumerate(zip(trans_data.T, new_points.T)):
    if data['s_0']['motion_planned']['position'].shape[0] == 36: # 30 hand + 3 pos + 3 ori of object
        initial_point = data['s_0']['motion_planned']['position'][30+i] # set the original position as initial sampled point position
    else:
        initial_point = None
    new_trans_data.append(interpolate_data(d, nd, initial_point))
new_trans_data = np.array(new_trans_data).T
print(new_trans_data.shape)


# new_ori_data = []
# for d in ori_data.T:
#     new_ori_data.append(interpolate_data(d))
# new_ori_data = np.array(new_ori_data).T
# print(new_ori_data.shape)
new_ori_data = ori_data

[[-0.21605517  0.10845744  0.04939834]
 [-0.21605517  0.10845744  0.44939834]
 [-0.21605517  0.30845744 -0.05060166]]
(76, 3)


In [22]:
import copy
save_dir = f'specified_trajs/{env}'
os.makedirs(save_dir, exist_ok=True)
new_traj = copy.copy(dict(data))

new_traj['object_translation'] = new_trans_data
new_traj['object_orientation'] = new_ori_data
# new_traj['SIM_SUBSTEPS'] = int(data['SIM_SUBSTEPS']/3)
new_traj['initial_translation_offset'] = offset
print(new_traj.keys(), new_traj['SIM_SUBSTEPS'])
np.savez(save_dir+f'/{env}', **new_traj)  # save a dict as npz

dict_keys(['s_0', 'object_translation', 'object_orientation', 'length', 'SIM_SUBSTEPS', 'DATA_SUBSTEPS', 'initial_translation_offset']) 3
